In [1]:
import json
import pandas as pd
import duckdb
import os

In [ ]:
import googlemaps
from datetime import datetime

def get_county_from_coordinates(lat, lon, api_key):
    # Create a client
    gmaps = googlemaps.Client(key=api_key)
    
    # Reverse geocode the coordinates
    result = gmaps.reverse_geocode((lat, lon))
    
    # Look for administrative_area_level_2 (county/alcaldía) in the results
    county = None
    if result:
        for component in result[0]['address_components']:
            if 'administrative_area_level_2' in component['types']:
                county = component['long_name']
                break
    
    return county

# Example usage:
# Replace with your actual API key
# Load environment variables from .env file
load_dotenv()
api_key = os.getenv('google_key')

# county = get_county_from_coordinates(19.4326, -99.1332, api_key)
# print(county)
# county = get_county_from_coordinates(19.4326, -99.1332, api_key)
# print(county)

for station_info in stations.to_dict('records')[0:1]:
     aah = get_county_from_coordinates(lat=station_info['lat_end'], lon=station_info['lon_end'], api_key=api_key)

In [ ]:
import googlemaps
from datetime import datetime

gmaps = googlemaps.Client(key=api_key)

all_results = []

for station_info in stations[['ciclo_estacionarribo','lat_end','lon_end']].drop_duplicates().to_dict('records')[0:10]:
    
    regions_df = []
    example = gmaps.reverse_geocode((station_info['lat_end'], station_info['lon_end']))

    for i in example[0]['address_components']:
        regions_df.append({
            "long_name": i['long_name'],
            "types": ",".join(i['types'])
        })
    
    result_df = pd.DataFrame(regions_df)
    result_df['ciclo_estacionarribo'] = station_info['ciclo_estacionarribo']

    all_results.append(result_df)

import time
e_list = []
for i,station_info in enumerate(stations[['ciclo_estacionarribo','lat_end','lon_end']].drop_duplicates().to_dict('records')):
    
    example = gmaps.reverse_geocode((station_info['lat_end'], station_info['lon_end']))
    export_json = {"ciclo_estacionarribo":station_info['ciclo_estacionarribo'],"googloe_maps_response": example}
    e_list.append(export_json)
    # To avoid hitting the API rate limit, we can add a sleep time
    if i % 30 == 0:
        print(f"Processed {i} stations, sleeping for 5 second to avoid rate limiting...")
        time.sleep(5)  # Sleep for 5 second between requests

# Save the results to a JSON file
with open('/Volumes/T7 Shield/Ecobici/stations_geocode_results.json', 'w') as f:
    json.dump(e_list, f, indent=4)
print(f"Saved geocoding results for {len(e_list)} stations to 'stations_geocode_results.json'.")

In [2]:
# Youtube  Keys
API_KEY = os.getenv('openai')

In [4]:
with open('/Volumes/T7 Shield/Ecobici/stations_geocode_results.json') as f:
    alcaldias = json.load(f)

In [5]:
# Create a dataframe with ciclo_estacionarribo and formatted_address
df = pd.DataFrame([
    {
        'ciclo_estacionarribo': item['ciclo_estacionarribo'],
        'formatted_address': item['googloe_maps_response'][0]['formatted_address']
    }
    for item in alcaldias
])

In [6]:
def get_alcaldia(address_components):
    for component in address_components:
        if 'sublocality_level_1' in component['types']:
            return component['long_name']
    return None

print(get_alcaldia(alcaldias[1]['googloe_maps_response'][0]['address_components']))

None


In [20]:
import openai
import math
import json
import tiktoken

def count_tokens(text, model="gpt-3.5-turbo"):
    """Count tokens for a given text using tiktoken."""
    encoding = tiktoken.encoding_for_model(model)
    return len(encoding.encode(text))

def extract_alcaldia_colonia_zip(addresses, api_key, max_tokens=4096, model="gpt-3.5-turbo"):
    """
    Extract alcaldia, colonia, and zip code from a list of addresses using OpenAI API.

    Parameters:
        addresses (list): List of addresses to process.
        api_key (str): Your OpenAI API key.
        max_tokens (int): Maximum token limit for the OpenAI model.
        model (str): OpenAI model to use.

    Returns:
        dict: {
            "results": {address: {"alcaldia": ..., "colonia": ..., "zip_code": ...}, ...},
            "token_summary": {
                "total_prompt_tokens": int,
                "total_response_tokens": int,
                "total_tokens_used": int,
                "total_tokens_left_on_table": int
            }
        }
    """

    client = openai.OpenAI(api_key=api_key)

    avg_tokens_per_address = 20
    max_addresses = math.floor((max_tokens - 500) / avg_tokens_per_address)

    results = {}

    # Token summary stats
    total_prompt_tokens = 0
    total_response_tokens = 0
    total_tokens_left = 0

    for i in range(0, len(addresses), max_addresses):
        batch = addresses[i:i + max_addresses]
        prompt = (
            "Extract the alcaldia, colonia, and zip code from the following addresses in JSON format. "
            "Each address should be a key, and the value should be a dictionary with keys 'alcaldia', 'colonia', and 'zip_code'.\n\n"
            + "\n".join(batch)
        )

        prompt_tokens = count_tokens(prompt, model)
        allowed_response_tokens = max_tokens - prompt_tokens

        print(f"\nBatch {i // max_addresses + 1}:")
        print(f"- Prompt tokens: {prompt_tokens}")
        print(f"- Max allowed response tokens: {allowed_response_tokens}")

        try:
            response = client.chat.completions.create(
                model=model,
                messages=[{"role": "user", "content": prompt}],
                max_tokens=allowed_response_tokens,
                temperature=0
            )

            response_content = response.choices[0].message.content
            response_tokens = count_tokens(response_content, model)
            unused_tokens = allowed_response_tokens - response_tokens

            print(f"- Response tokens: {response_tokens}")
            print(f"- Tokens left on table: {unused_tokens}")
            print(f"- Total used: {prompt_tokens + response_tokens}")

            # Tally
            total_prompt_tokens += prompt_tokens
            total_response_tokens += response_tokens
            total_tokens_left += unused_tokens

            batch_results = json.loads(response_content)
            results.update(batch_results)

        except Exception as e:
            print(f"Error processing batch: {e}")

    return {
        "results": results,
        "token_summary": {
            "total_prompt_tokens": total_prompt_tokens,
            "total_response_tokens": total_response_tokens,
            "total_tokens_used": total_prompt_tokens + total_response_tokens,
            "total_tokens_left_on_table": total_tokens_left
        }
    }

# Example usage
results0_10 = extract_alcaldia_colonia_zip(df['formatted_address'].to_list()[0:10],API_KEY)


Batch 1:
- Prompt tokens: 386
- Max allowed response tokens: 3710
- Response tokens: 712
- Tokens left on table: 2998
- Total used: 1098


In [85]:
result1 = [{'alcaldia':v['alcaldia'], 'colonia':v['colonia'], 'zip_code':v['zip_code']} for v in results0_10['results'].values()]

In [21]:
results10_30 = extract_alcaldia_colonia_zip(df['formatted_address'].to_list()[10:30],API_KEY)


Batch 1:
- Prompt tokens: 801
- Max allowed response tokens: 3295
- Response tokens: 1513
- Tokens left on table: 1782
- Total used: 2314


In [86]:
result2 = [{'alcaldia':v['alcaldia'], 'colonia':v['colonia'], 'zip_code':v['zip_code']} for v in results10_30['results'].values()]

In [23]:
results30_60 = extract_alcaldia_colonia_zip(df['formatted_address'].to_list()[30:60],API_KEY)


Batch 1:
- Prompt tokens: 1070
- Max allowed response tokens: 3026
- Response tokens: 2152
- Tokens left on table: 874
- Total used: 3222


In [87]:
result3 = [{'alcaldia':v['alcaldia'], 'colonia':v['colonia'], 'zip_code':v['zip_code']} for v in results30_60['results'].values()]

In [24]:
results60_90 = extract_alcaldia_colonia_zip(df['formatted_address'].to_list()[60:90],API_KEY)


Batch 1:
- Prompt tokens: 1091
- Max allowed response tokens: 3005
- Response tokens: 2206
- Tokens left on table: 799
- Total used: 3297


In [88]:
result4 = [{'alcaldia':v['alcaldia'], 'colonia':v['colonia'], 'zip_code':v['zip_code']} for v in results60_90['results'].values()]

In [74]:
kk= []
for i in range(90, len(df['formatted_address']), 30):
    results109 = df[['ciclo_estacionarribo']].iloc[i:i+30,:].reset_index()
    kk.append(results109)

In [77]:
stations_index = pd.concat(kk).drop(columns='index').reset_index(drop=True)

In [36]:
results_list = []

for i in range(90, len(df['formatted_address']), 30):
    print(i,i+30)
    results = extract_alcaldia_colonia_zip(df['formatted_address'].to_list()[i:i+30], API_KEY)
    results_list.append(results['results'])


90 120

Batch 1:
- Prompt tokens: 1100
- Max allowed response tokens: 2996
- Response tokens: 2202
- Tokens left on table: 794
- Total used: 3302
120 150

Batch 1:
- Prompt tokens: 1151
- Max allowed response tokens: 2945
- Response tokens: 2263
- Tokens left on table: 682
- Total used: 3414
150 180

Batch 1:
- Prompt tokens: 1141
- Max allowed response tokens: 2955
- Response tokens: 2260
- Tokens left on table: 695
- Total used: 3401
180 210

Batch 1:
- Prompt tokens: 1089
- Max allowed response tokens: 3007
- Response tokens: 2190
- Tokens left on table: 817
- Total used: 3279
210 240

Batch 1:
- Prompt tokens: 1140
- Max allowed response tokens: 2956
- Response tokens: 2241
- Tokens left on table: 715
- Total used: 3381
240 270

Batch 1:
- Prompt tokens: 1137
- Max allowed response tokens: 2959
- Response tokens: 2250
- Tokens left on table: 709
- Total used: 3387
270 300

Batch 1:
- Prompt tokens: 1169
- Max allowed response tokens: 2927
- Response tokens: 2277
- Tokens left on ta

In [50]:
### How to save the results to a json file
with open('/Volumes/T7 Shield/Ecobici/alcaldias_colonias_zips.json', 'w') as f:
    json.dump(results_list, f)

In [80]:
# Extract alcaldia, colonia, and zip_code
loo = []
for i in range(0, len(results_list)):
    result = [{'alcaldia':v['alcaldia'], 'colonia':v['colonia'], 'zip_code':v['zip_code']} for v in results_list[i].values()]
    loo.append(pd.DataFrame(result))

In [82]:
stations_final = pd.concat(loo).reset_index(drop=True)

In [89]:
a = pd.concat([df[['ciclo_estacionarribo']].iloc[0:10,:], pd.DataFrame(result1)], axis=1)
b = pd.concat([df[['ciclo_estacionarribo']].iloc[10:30,:].reset_index(), pd.DataFrame(result2)], axis=1)
c = pd.concat([df[['ciclo_estacionarribo']].iloc[30:60,:].reset_index(), pd.DataFrame(result3)], axis=1)
d = pd.concat([df[['ciclo_estacionarribo']].iloc[60:90,:].reset_index(), pd.DataFrame(result4)], axis=1)

In [90]:
all_stations = pd.concat([stations_index, stations_final], axis=1)
stations__final = pd.concat([a,b,c,d,all_stations], axis=0)

In [92]:
stations__final.to_csv('/Volumes/T7 Shield/Ecobici/stations_alcaldias_colonias_zips.csv', index=False)